In [66]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [67]:
from lib.reproduction import major_oxides
from sklearn.metrics import mean_squared_error
from lib import full_flow_dataloader
import mlflow
import numpy as np
import datetime
import os
os.environ["KERAS_BACKEND"] = "torch"

import torch
import keras


torch.manual_seed(42)
np.random.seed(42)


In [68]:
print(keras.__version__)

3.2.1


In [69]:
import torch.nn as nn
import torch.optim as optim

# Check if GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [70]:
train_processed, test_processed = full_flow_dataloader.load_full_flow_data(load_cache_if_exits=True, average_shots=True)

In [78]:
# cnn_regression_optimized.py
from keras import layers, optimizers, regularizers, Model

def build_model(input_dim, output_dim):
    inputs = layers.Input(shape=(input_dim,))
    x = layers.Reshape((48, 128, 1))(inputs)

    # Initial convolutional layer
    x = layers.Conv2D(64, (7, 7), strides=(2, 2), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    # Residual block 1
    shortcut1 = x
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Add()([x, shortcut1])
    x = layers.Activation('relu')(x)

    # Residual block 2
    shortcut2 = x
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Add()([x, shortcut2])
    x = layers.Activation('relu')(x)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)

    # Using OUTPUT_DIM parallel layers to output the target value, then concatenate
    outputs = [layers.Dense(1, kernel_regularizer=regularizers.l2(0.01))(x) for i in range(output_dim)]
    if len(outputs) > 1:
        outputs = layers.Concatenate()(outputs)
    else:
        outputs = outputs[0]  # If only one output, no need to concatenate

    model = Model(inputs=inputs, outputs=outputs)

    # Optimizer with a custom learning rate
    optimizer = optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='mse', metrics=['root_mean_squared_error', 'mae'])
    return model

# Constants
INPUT_DIM = 6144  # Number of features per sample
OUTPUT_DIM = 8    # Number of continuous values as output

# Model Creation
model = build_model(INPUT_DIM, OUTPUT_DIM)
model.summary()


ValueError: Inputs have incompatible shapes. Received shapes (6, 16, 128) and (12, 32, 64)

In [ ]:
drop_cols = major_oxides + ["ID", "Sample Name"]

X_train = train_processed.drop(columns=drop_cols)
y_train = train_processed[major_oxides]

X_test = test_processed.drop(columns=drop_cols)
y_test = test_processed[major_oxides]


In [ ]:
X_train_reshaped = X_train.to_numpy().reshape(-1, 6144, 1)
X_test_reshaped = X_test.to_numpy().reshape(-1, 6144, 1)

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

In [ ]:
def run_cnn_experiment(
    X_train: np.ndarray,
    y_train: np.ndarray,
    X_test: np.ndarray,
    y_test: np.ndarray,
    model: keras.Model,
    epochs: int,
    batch_size: int,
    callbacks: list = [],
    major_oxides: list = [],
):
    with mlflow.start_run(run_name="CNN"):
        model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1, callbacks=callbacks)
        y_pred = model.predict(X_test)
        for i, oxide in enumerate(major_oxides):
            y_test_oxide = y_test[:, i]
            y_pred_oxide = y_pred[:, i]
            rmse = mean_squared_error(y_test_oxide, y_pred_oxide, squared=False)
            mlflow.log_metric(f"rmse_{oxide}", float(rmse))

In [ ]:
callback = keras.callbacks.EarlyStopping(monitor="val_loss", patience=6, restore_best_weights=True)

class MLFlowCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs is not None:
            for key, value in logs.items():
                mlflow.log_metric(f"{key}", value, step=epoch)


mlflow.set_experiment(f'CNN_Res_{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}')
run_cnn_experiment(
    X_train_reshaped,
    y_train.to_numpy(),
    X_test_reshaped,
    y_test.to_numpy(),
    model,
    epochs=1000,
    batch_size=32,
    callbacks=[MLFlowCallback(), callback],
    major_oxides=major_oxides,
)



2024/04/19 11:12:49 INFO mlflow.tracking.fluent: Experiment with name 'CNN_Res_20240419-111245' does not exist. Creating a new experiment.


Epoch 1/1000
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - loss: 335.4956 - mean_absolute_error: 9.7895 - val_loss: 280.2798 - val_mean_absolute_error: 10.1345
Epoch 2/1000
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - loss: 35.9537 - mean_absolute_error: 4.3526 - val_loss: 238.6214 - val_mean_absolute_error: 9.8099
Epoch 3/1000
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - loss: 29.2822 - mean_absolute_error: 3.7743 - val_loss: 88.1235 - val_mean_absolute_error: 4.9903
Epoch 4/1000
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - loss: 24.0560 - mean_absolute_error: 3.3637 - val_loss: 90.7203 - val_mean_absolute_error: 5.0241
Epoch 5/1000
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - loss: 22.1862 - mean_absolute_error: 3.1414 - val_loss: 84.3762 - val_mean_absolute_error: 4.7604
Epoch 6/1000
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - loss: 19.6694 - mean_absolute_error: 2.9276 - val_loss: 69.7693 - val_mean_absolute_error: 4.1859
Epoch 7/1000
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - loss: 17.8358 - mean_ab

/home/christian/projects/p9/baseline/venv/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/christian/projects/p9/baseline/venv/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/christian/projects/p9/baseline/venv/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/christian/projects/p9/baseline/venv/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squa